### References

[video](https://www.youtube.com/watch?v=qU7wO02urYU)

In [1]:
from datasets import load_dataset
import torch
from transformers import ViTImageProcessor, TrainingArguments, ViTForImageClassification, Trainer
import os

# model id to be used
model_id = 'google/vit-base-patch16-224-in21k'

# device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
dataset_train = load_dataset("imagefolder","PlantVillage", split="train") # streaming=True for lazy loading

dataset_train, dataset_train.features

Resolving data files:   0%|          | 0/20402 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/576 [00:00<?, ?it/s]

(Dataset({
     features: ['image', 'label'],
     num_rows: 20401
 }),
 {'image': Image(mode=None, decode=True, id=None),
  'label': ClassLabel(names=['00 Chilli - Healthy', '01 Chilli - Leaf Curl Virus', '02 Pepper Bell - Bacterial Spot', '03 Pepper Bell - Healthy', '04 Potato - Early Blight', '05 Potato - Healthy', '06 Potato - Late Blight', '07 Tomato - Bacterial Spot', '08 Tomato - Early Blight', '09 Tomato - Healthy', '10 Tomato - Late Blight', '11 Tomato - Leaf Mold', '12 Tomato - Mosaic Virus', '13 Tomato - Septoria Leaf Spot', '14 Tomato - Target Spot', '15 Tomato - Two Spotted Spider Mite', '16 Tomato - Yellow Leaf Curl Virus'], id=None)})

In [3]:
dataset_test = load_dataset("imagefolder", "PlantVillage", split="test")

dataset_test

Resolving data files:   0%|          | 0/20402 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/576 [00:00<?, ?it/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 576
})

In [4]:
dataset_train[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=200x200>,
 'label': 0}

In [5]:
feature_extractor = ViTImageProcessor.from_pretrained(
    model_id,
    ignore_mismatched_sizes=True
)
feature_extractor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [6]:
def preprocess(batch):
    # take a list of PIL images and turn them to pixel values
    images = [img.convert('RGB') if img.mode != 'RGB' else img for img in batch['image']]
    inputs = feature_extractor(
        images,
        return_tensors='pt'
    )
    # include the labels
    inputs['label'] = batch['label']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [7]:
prepared_train = dataset_train.with_transform(preprocess)
prepared_test = dataset_test.with_transform(preprocess)

In [8]:
labels = dataset_train.features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_id,
    ignore_mismatched_sizes= True,
    num_labels=len(labels),
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)}
)
model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [9]:
training_args = TrainingArguments(
    output_dir="./plantvillage_model",
    per_device_train_batch_size=16,
    eval_strategy="steps",
    num_train_epochs=3,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=prepared_train,
    eval_dataset=prepared_test,
    tokenizer=feature_extractor,
)

/home/dlsathvik04/miniconda3/envs/PT/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
if os.path.exists("./plantvillage_checkpoints"):
    # Get the latest checkpoint
    checkpoints = [dir for dir in os.listdir("./plantvillage_checkpoints") if dir.startswith("checkpoint-")]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
        checkpoint_path = os.path.join("./plantvillage_checkpoints", latest_checkpoint)
        print(f"Resuming training from checkpoint: {checkpoint_path}")
        # Resume training
        train_results = trainer.train(resume_from_checkpoint=checkpoint_path)
    else:
        print("No checkpoint found. Starting training from scratch.")
        train_results = trainer.train()
else:
    print("No checkpoint directory found. Starting training from scratch.")
    train_results = trainer.train()

No checkpoint directory found. Starting training from scratch.


  0%|          | 0/3828 [00:00<?, ?it/s]

{'loss': 2.5494, 'grad_norm': 1.5736089944839478, 'learning_rate': 0.00019947753396029258, 'epoch': 0.01}
{'loss': 2.1203, 'grad_norm': 1.9132421016693115, 'learning_rate': 0.00019895506792058518, 'epoch': 0.02}
{'loss': 1.7233, 'grad_norm': 2.2865209579467773, 'learning_rate': 0.00019843260188087775, 'epoch': 0.02}
{'loss': 1.3617, 'grad_norm': 1.8445883989334106, 'learning_rate': 0.00019791013584117032, 'epoch': 0.03}
{'loss': 1.088, 'grad_norm': 1.5586708784103394, 'learning_rate': 0.0001973876698014629, 'epoch': 0.04}
{'loss': 1.0526, 'grad_norm': 2.6481359004974365, 'learning_rate': 0.0001968652037617555, 'epoch': 0.05}
{'loss': 0.7676, 'grad_norm': 3.6279137134552, 'learning_rate': 0.00019634273772204807, 'epoch': 0.05}
{'loss': 0.8775, 'grad_norm': 2.0404765605926514, 'learning_rate': 0.00019582027168234064, 'epoch': 0.06}
{'loss': 0.7132, 'grad_norm': 1.008786678314209, 'learning_rate': 0.00019529780564263324, 'epoch': 0.07}
{'loss': 0.5184, 'grad_norm': 2.7155203819274902, 'le

  0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.811065673828125, 'eval_runtime': 7.9477, 'eval_samples_per_second': 72.474, 'eval_steps_per_second': 9.059, 'epoch': 0.08}
{'loss': 0.5544, 'grad_norm': 1.9914830923080444, 'learning_rate': 0.0001942528735632184, 'epoch': 0.09}
{'loss': 0.5146, 'grad_norm': 4.156414031982422, 'learning_rate': 0.00019373040752351098, 'epoch': 0.09}
{'loss': 0.5305, 'grad_norm': 2.6715192794799805, 'learning_rate': 0.00019320794148380358, 'epoch': 0.1}
{'loss': 0.389, 'grad_norm': 0.7182912826538086, 'learning_rate': 0.00019268547544409615, 'epoch': 0.11}
{'loss': 0.3797, 'grad_norm': 0.4292014241218567, 'learning_rate': 0.00019216300940438872, 'epoch': 0.12}
{'loss': 0.4057, 'grad_norm': 0.8226805329322815, 'learning_rate': 0.00019164054336468132, 'epoch': 0.13}
{'loss': 0.3866, 'grad_norm': 2.682779312133789, 'learning_rate': 0.0001911180773249739, 'epoch': 0.13}
{'loss': 0.2756, 'grad_norm': 0.31661897897720337, 'learning_rate': 0.00019059561128526647, 'epoch': 0.14}
{'loss': 0.3227, '

...

{'eval_loss': 0.0529453344643116, 'eval_runtime': 7.6328, 'eval_samples_per_second': 75.464, 'eval_steps_per_second': 9.433, 'epoch': 2.82}
{'loss': 0.0011, 'grad_norm': 0.00280874059535563, 'learning_rate': 1.1389759665621736e-05, 'epoch': 2.83}
{'loss': 0.0226, 'grad_norm': 0.012960979714989662, 'learning_rate': 1.0867293625914315e-05, 'epoch': 2.84}
{'loss': 0.0011, 'grad_norm': 0.0036664933431893587, 'learning_rate': 1.0344827586206897e-05, 'epoch': 2.84}
{'loss': 0.0095, 'grad_norm': 0.0030029420740902424, 'learning_rate': 9.822361546499479e-06, 'epoch': 2.85}
{'loss': 0.0013, 'grad_norm': 0.002860903274267912, 'learning_rate': 9.299895506792059e-06, 'epoch': 2.86}
{'loss': 0.015, 'grad_norm': 0.0090488838031888, 'learning_rate': 8.77742946708464e-06, 'epoch': 2.87}
{'loss': 0.001, 'grad_norm': 0.002948290202766657, 'learning_rate': 8.254963427377221e-06, 'epoch': 2.88}
{'loss': 0.0011, 'grad_norm': 0.005187514238059521, 'learning_rate': 7.732497387669801e-06, 'epoch': 2.88}
{'los

  0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.05453566834330559, 'eval_runtime': 7.6206, 'eval_samples_per_second': 75.585, 'eval_steps_per_second': 9.448, 'epoch': 2.9}
{'loss': 0.001, 'grad_norm': 0.0026585387531667948, 'learning_rate': 6.165099268547545e-06, 'epoch': 2.91}
{'loss': 0.0013, 'grad_norm': 0.0026715446729213, 'learning_rate': 5.642633228840126e-06, 'epoch': 2.92}
{'loss': 0.0011, 'grad_norm': 0.002514324616640806, 'learning_rate': 5.120167189132706e-06, 'epoch': 2.92}
{'loss': 0.0267, 'grad_norm': 0.00951069314032793, 'learning_rate': 4.5977011494252875e-06, 'epoch': 2.93}
{'loss': 0.0015, 'grad_norm': 0.004121039062738419, 'learning_rate': 4.075235109717869e-06, 'epoch': 2.94}
{'loss': 0.0011, 'grad_norm': 0.003485678927972913, 'learning_rate': 3.5527690700104498e-06, 'epoch': 2.95}
{'loss': 0.0012, 'grad_norm': 0.003161568893119693, 'learning_rate': 3.0303030303030305e-06, 'epoch': 2.95}
{'loss': 0.0322, 'grad_norm': 5.592681884765625, 'learning_rate': 2.507836990595611e-06, 'epoch': 2.96}
{'loss'

  0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.05512528494000435, 'eval_runtime': 7.6198, 'eval_samples_per_second': 75.592, 'eval_steps_per_second': 9.449, 'epoch': 2.98}
{'loss': 0.0083, 'grad_norm': 0.002920000348240137, 'learning_rate': 9.404388714733543e-07, 'epoch': 2.99}
{'loss': 0.0011, 'grad_norm': 0.00848575308918953, 'learning_rate': 4.179728317659353e-07, 'epoch': 2.99}
{'train_runtime': 1472.1349, 'train_samples_per_second': 41.574, 'train_steps_per_second': 2.6, 'train_loss': 0.10247549182473115, 'epoch': 3.0}


In [11]:
trainer.save_model("./plantvillage_final_model")
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =          3.0
  total_flos               = 4417616739GF
  train_loss               =       0.1025
  train_runtime            =   0:24:32.13
  train_samples_per_second =       41.574
  train_steps_per_second   =          2.6


In [ ]:
dataset_test = load_dataset("imagefolder", "PlantVillage", split="test")
# model_name_or_path = './plantvillage_model/checkpoint-1300/'
model_name_or_path = './plantvillage_final_model/'
model_finetuned = ViTForImageClassification.from_pretrained(model_name_or_path)
# import features
feature_extractor_finetuned = ViTImageProcessor.from_pretrained(model_name_or_path)

In [ ]:
model_finetuned.eval()

# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels = []

missed = []

# Iterate through the test dataset
for sample in dataset_test:
    image = sample['image']
    label = sample['label']

    # Extract features from the image
    inputs = feature_extractor_finetuned(image, return_tensors="pt")

    # Get model predictions
    with torch.no_grad():
        logits = model_finetuned(**inputs).logits
    
    # Get the predicted label
    predicted_label = logits.argmax(-1).item()
    # print(predicted_label)

    if (label != predicted_label):
        missed.append((image,label,))
    
    # Append the true and predicted labels
    true_labels.append(label)
    predicted_labels.append(predicted_label)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

In [ ]:
# Get the label names
label_names = dataset_test.features['label'].names

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')

# Show the plot
plt.show()